In [175]:
import pandas as pd

In [176]:
df_train = pd.read_csv("../data/train_users_2.csv")
df_test = pd.read_csv("../data/test_users.csv")
df_all = pd.concat([df_train, df_test])

In [177]:
df_target = df_train["country_destination"]

In [178]:
df_all = df_all.drop("age", axis = 1)
df_all = df_all.drop("date_first_booking", axis = 1)

In [179]:
df_all["first_affiliate_tracked"].fillna("untracked", inplace = True)

In [180]:
df_all.isnull().sum()

affiliate_channel              0
affiliate_provider             0
country_destination        62096
date_account_created           0
first_affiliate_tracked        0
first_browser                  0
first_device_type              0
gender                         0
id                             0
language                       0
signup_app                     0
signup_flow                    0
signup_method                  0
timestamp_first_active         0
dtype: int64

In [181]:
df_all["date_account_created"] = pd.to_datetime(df_all["date_account_created"], format = "%Y-%m-%d")

In [182]:
df_all["timestamp_first_active"] = pd.to_datetime(df_all["timestamp_first_active"], format="%Y%m%d%H%M%S", errors='ignore')

In [183]:
df_all["ac_year"] = df_all["date_account_created"].apply(lambda x : x.year)
df_all["ac_month"] = df_all["date_account_created"].apply(lambda x : x.month)
df_all["ac_day"] = df_all["date_account_created"].apply(lambda x : x.day)

In [184]:
df_all["fa_year"] = df_all["timestamp_first_active"].apply(lambda x : x.year)
df_all["fa_month"] = df_all["timestamp_first_active"].apply(lambda x : x.month)
df_all["fa_day"] = df_all["timestamp_first_active"].apply(lambda x : x.day)
df_all["fa_hour"] = df_all["timestamp_first_active"].apply(lambda x : x.hour)
df_all["fa_minute"] = df_all["timestamp_first_active"].apply(lambda x : x.minute)
df_all["fa_second"] = df_all["timestamp_first_active"].apply(lambda x : x.second)

In [185]:
df_all = df_all.drop("date_account_created", axis = 1)
df_all = df_all.drop("timestamp_first_active", axis = 1)
df_all = df_all.drop("country_destination", axis = 1)

In [186]:
df_all = df_all.drop("id", axis = 1)

In [187]:
df_all = pd.get_dummies(df_all)

In [188]:
df_train = df_all.iloc[:213451]
df_test = df_all.iloc[213451:]

In [189]:
df_train.shape, df_test.shape

((213451, 145), (62096, 145))

# 1. QDA 평가

In [190]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [195]:
y = df_target
X = df_train
X_test = df_test

In [196]:
qda = QuadraticDiscriminantAnalysis(store_covariance=True).fit(X, y)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [197]:
X.shape, X_test.shape, df_target.shape

((213451, 145), (62096, 145), (213451,))

In [204]:
y_pre = qda.predict(X)

In [211]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [212]:
confusion_matrix(y, y_pre)

array([[    17,      1,      1,      0,      0,      0,      0,      0,
             6,    514,      0,      0],
       [    30,     24,      6,      0,      0,      2,      0,      0,
            21,   1345,      0,      0],
       [    25,      4,     11,      0,      0,      0,      0,      0,
            19,   1002,      0,      0],
       [    41,     10,     14,      0,      2,      4,      2,      0,
            32,   2142,      1,      1],
       [   101,     17,     30,      0,      7,      9,      2,      0,
            64,   4790,      0,      3],
       [    39,     12,      5,      0,      0,     15,      0,      0,
            24,   2229,      0,      0],
       [    45,     10,     17,      0,      1,      6,      5,      0,
            53,   2697,      0,      1],
       [  3374,   1194,   1522,      2,     71,    245,    132,    124,
          4776, 112914,     64,    125],
       [    16,      1,      2,      0,      0,      0,      0,      0,
            24,    719, 

In [214]:
print(classification_report(y, y_pre))

             precision    recall  f1-score   support

         AU       0.00      0.03      0.01       539
         CA       0.01      0.02      0.01      1428
         DE       0.01      0.01      0.01      1061
         ES       0.00      0.00      0.00      2249
         FR       0.06      0.00      0.00      5023
         GB       0.03      0.01      0.01      2324
         IT       0.03      0.00      0.00      2835
        NDF       0.89      0.00      0.00    124543
         NL       0.00      0.03      0.01       762
         PT       0.00      1.00      0.00       217
         US       0.50      0.00      0.00     62376
      other       0.13      0.00      0.00     10094

avg / total       0.67      0.00      0.00    213451

